### Loading the dataset into DataFrames

In [1]:
import os
from datasets import load_dataset

import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint

import pandas as pd
import torch

from local_dataset_utilities import download_dataset, load_dataset_into_to_dataframe, partition_dataset
from local_dataset_utilities import IMDBDataset

In [2]:
if not torch.cuda.is_available():
    print("Please switch to a GPU machine before running this notebook.")

In [3]:
files = ("test.csv", "train.csv", "val.csv")
download = True

for f in files:
    if not os.path.exists(os.path.join("data", f)):
        download = False

if download is False:
    download_dataset()
    df = load_dataset_into_to_dataframe()
    partition_dataset(df)

In [4]:
df_train = pd.read_csv(os.path.join("data", "train.csv"))
df_val = pd.read_csv(os.path.join("data", "val.csv"))
df_test = pd.read_csv(os.path.join("data", "test.csv"))

### Tokenization and Numericalization

In [5]:
imdb_dataset = load_dataset(
    "csv",
    data_files={
        "train": os.path.join("data", "train.csv"),
        "validation": os.path.join("data", "val.csv"),
        "test": os.path.join("data", "test.csv"),
    },
)

print(imdb_dataset)

DatasetDict({
    train: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 35000
    })
    validation: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['index', 'text', 'label'],
        num_rows: 10000
    })
})


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def tokenize_text(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

In [8]:
imdb_tokenized = imdb_dataset.map(tokenize_text, batched=True, batch_size=None)

In [9]:
del imdb_dataset

In [10]:
imdb_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [11]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"

### Set Up DataLoaders

In [12]:
from torch.utils.data import DataLoader, Dataset


class IMDBDataset(Dataset):
    def __init__(self, dataset_dict, partition_key="train"):
        self.partition = dataset_dict[partition_key]

    def __getitem__(self, index):
        return self.partition[index]

    def __len__(self):
        return self.partition.num_rows

In [13]:
train_dataset = IMDBDataset(imdb_tokenized, partition_key="train")
val_dataset = IMDBDataset(imdb_tokenized, partition_key="validation")
test_dataset = IMDBDataset(imdb_tokenized, partition_key="test")

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=12,
    shuffle=True, 
    num_workers=4
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=12,
    num_workers=4
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=12,
    num_workers=4
)

### Initializing DistilBERT

In [14]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, requires_grad: {param.requires_grad}")

Parameter: distilbert.embeddings.word_embeddings.weight, requires_grad: True
Parameter: distilbert.embeddings.position_embeddings.weight, requires_grad: True
Parameter: distilbert.embeddings.LayerNorm.weight, requires_grad: True
Parameter: distilbert.embeddings.LayerNorm.bias, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.q_lin.weight, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.q_lin.bias, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.k_lin.weight, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.k_lin.bias, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.v_lin.weight, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.v_lin.bias, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.out_lin.weight, requires_grad: True
Parameter: distilbert.transformer.layer.0.attention.out_lin.bias, requires_grad: True
Parameter: distilb

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total number of trainable parameters:", count_parameters(model))

Total number of trainable parameters: 66955010


### Finetuning

In [17]:
from local_model_utilities import CustomLightningModule

lightning_model = CustomLightningModule(model)

In [18]:
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger


callbacks = [
    ModelCheckpoint(
        save_top_k=1, mode="max", monitor="val_acc"
    )  # save top 1 model
]
logger = CSVLogger(save_dir="logs/", name="my-model")

In [19]:
trainer = L.Trainer(
    max_epochs=3,
    callbacks=callbacks,
    accelerator="gpu",
    precision="16-mixed",
    devices=1,
    logger=logger,
    log_every_n_steps=10,
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
import time
start = time.time()

trainer.fit(model=lightning_model,
            train_dataloaders=train_loader,
            val_dataloaders=val_loader)

end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type                                | Params | Mode 
-------------------------------------------------------------------------
0 | model    | DistilBertForSequenceClassification | 67.0 M | eval 
1 | val_acc  | MulticlassAccuracy                  | 0      | train
2 | test_acc | MulticlassAccuracy                  | 0      | train
-------------------------------------------------------------------------
67.0 M    Trainable params
0         Non-trainable params
67.0 M    Total params
267.820   Total estimated model params size (MB)
2         Modules in train mode
96        M

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


Time elapsed 12.33 min


In [21]:
train_acc = trainer.test(lightning_model, dataloaders=train_loader, ckpt_path="best", verbose=False)
val_acc = trainer.test(lightning_model, dataloaders=val_loader, ckpt_path="best", verbose=False)
test_acc = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best", verbose=False)

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt
/root/miniconda3/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:475: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at logs/my-model/version_0/checkpoints/epoch=0-step=2917.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

In [22]:
print(f"Train acc: {train_acc[0]['accuracy']*100:2.2f}%")
print(f"Val acc:   {val_acc[0]['accuracy']*100:2.2f}%")
print(f"Test acc:  {test_acc[0]['accuracy']*100:2.2f}%")

Train acc: 96.28%
Val acc:   92.32%
Test acc:  91.82%


In [23]:
import shutil

# Cleanup checkpoint files as we don't need them later
log_dir = f"logs/my-model"
if os.path.exists(log_dir):
    shutil.rmtree(log_dir)